# Fetching JWST Level 1b Files with Astroquery

This notebook will acquaint you with using a python package to query MAST/CAOM to fetch JWST L-1b files. To run this notebook you will need a reasonably recent version of Python (v3.7+) and of astroquery.mast (v0.4.6) package. Note that this is the most reliable way to find L-1b files, as they are not visibile using a SI Keyword search if higher-level products for the same observation exist in the archive. 

In [4]:
from astroquery.mast import Observations

## Observation Query

Construct a parameter query, which is equivalent to an Advanced Search in the MAST Portal. The set of all available parameters for a CAOM query is here.  Consider a search for NIRCam observations in program 1134. Use the **.query_criteria()** method.

In [5]:
# JR changed the PID to be NRC-3 NIRCam functional checkout
obs = Observations.query_criteria(
        obs_collection = 'JWST',
        proposal_id = '01448')
        #)  
# See if the query matched any observations
len(obs)

160

In [3]:
obs

dataproduct_type,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_max,t_exptime,wavelength_region,filters,em_min,em_max,target_classification,obs_title,t_obs_release,instrument_name,proposal_pi,proposal_id,proposal_type,project,sequence_number,provenance_name,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,intentType,obsid,objID
str5,int64,str4,str36,str27,float64,float64,float64,float64,float64,str8,str5,float64,float64,str1,str37,float64,str6,str11,str4,str4,str4,int64,str7,str1934,str62,str63,str6,bool,float64,str7,str8,str9
image,3,JWST,jw01448-o002_t014_nircam_clear-f277w,UNKNOWN,260.0,29.98333333333333,59698.27222111632,59698.27955254629,450.945,Infrared,F277W,2416.0,3127.0,--,Stray Light Pointed Model Correlation,59774.8541666,NIRCAM,"Smith, Erin",1448,JWST,JWST,--,CALJWST,POLYGON -100.00266979000003 29.95599688 -99.970725360000017 29.97890701 -99.970788183222879 29.978971841023771 -99.970582200000024 29.97911951 -99.970644934210384 29.979184249354443 -99.970438919999992 29.97933194 -99.99740678 30.00715121 -100.02894437999998 29.98339824 -100.02888524069975 29.9833381976637 -100.02908753000003 29.98318578 -100.02902826248453 29.983125607421933 -100.02923059 29.98297316 -100.00266979000003 29.95599688 -100.00266979000003 29.95599688,mast:JWST/product/jw01448-o002_t014_nircam_clear-f277w_i2d.jpg,mast:JWST/product/jw01448-o002_t014_nircam_clear-f277w_i2d.fits,PUBLIC,False,nan,science,82502385,168268706
image,3,JWST,jw01448-o008_t016_nircam_clear-f277w,UNKNOWN,277.0,-11.0,59698.38574840799,59698.3930805787,450.945,Infrared,F277W,2416.0,3127.0,--,Stray Light Pointed Model Correlation,59774.8541666,NIRCAM,"Smith, Erin",1448,JWST,JWST,--,CALJWST,POLYGON -83.01595063000002 -11.02251115 -82.979865060000009 -11.01649114 -82.9798801402817 -11.016407588666567 -82.97964771 -11.0163688 -82.979662681166658 -11.016285850442813 -82.979429830000015 -11.01624699 -82.98588503000002 -10.9804782 -83.022079220000023 -10.98739935 -83.022064699653029 -10.987476492394709 -83.022296910000023 -10.98752088 -83.022282197722532 -10.987599045035605 -83.022514440000009 -10.98764344 -83.01595063000002 -11.02251115 -83.01595063000002 -11.02251115,mast:JWST/product/jw01448-o008_t016_nircam_clear-f277w_i2d.jpg,mast:JWST/product/jw01448-o008_t016_nircam_clear-f277w_i2d.fits,PUBLIC,False,nan,science,82523890,168268725
image,3,JWST,jw01448-o008_t016_nircam_clear-f115w,UNKNOWN,277.0,-11.0,59698.40749505151,59698.41482796297,1803.78,Infrared,F115W,1013.0000000000001,1282.0,--,Stray Light Pointed Model Correlation,59774.8541666,NIRCAM,"Smith, Erin",1448,JWST,JWST,--,CALJWST,POLYGON -83.000702129999979 -11.0008759 -82.982822199999987 -10.99768592 -82.98283710136009 -10.997605414601976 -82.982604509999987 -10.99756391 -82.982619442337992 -10.997483238479454 -82.982386940000026 -10.99744175 -82.985648509999976 -10.97982003 -83.003575459999979 -10.9832711 -83.003560698895669 -10.983348557796058 -83.003793050000013 -10.98339328 -83.003778329828791 -10.983470522769819 -83.004010709999989 -10.98351525 -83.000702129999979 -11.0008759 -83.000702129999979 -11.0008759 POLYGON -83.01945035 -11.00421566 -83.001864450000028 -11.00101681 -83.0018796619102 -11.000937213981906 -83.00164675000002 -11.00089484 -83.001662004954582 -11.000815019716388 -83.00142917 -11.00077266 -83.00474392000001 -10.98342742 -83.022384719999991 -10.9868304 -83.022369524997529 -10.986907699812752 -83.022602319999976 -10.9869526 -83.022587180367992 -10.987029617676539 -83.022819990000016 -10.98707452 -83.01945035 -11.00421566 -83.01945035 -11.00421566 POLYGON -82.99743106 -11.01937704 -82.979531969999982 -11.01655681 -82.979546556835231 -11.016471392236348 -82.979314240000008 -11.01643478 -82.979328853389987 -11.016349207924284 -82.97909669 -11.01631262 -82.982100150000008 -10.99872399 -82.999997370000017 -11.00179338 -82.9999831021622 -11.001875806437997 -83.000214939999978 -11.00191556 -83.000200711284009 -11.001997759832621 -83.00043265 -11.00203753 -82.99743106 -11.01937704 -82.

Note that the above query can be modified to search for multiple program IDs by making the value of the **proposal_id** parameter a list of IDs. The result is an astropy Table object, with one row per matched observation. 

<div class="alert alert-block alert-info">

<span style="color:black">
    If for a particular query you are not interested in observations from any mission except JWST, specify it with the parameter obs_collection='JWST'. This will narrow the list of possible matches considerably, and speed up your query.
    
</span>
</div>

## Data Product Query

Having found Observations that match your criteria, the next step is to fetch the list of data products associated with each Observation. The following call returns a table of all products, one per row, which could number in the hundreds or thousands of files, including guide-star and ancillary files. We will filter the list later.

Query for the products by passing in the table of Observations.

For the stray light CAR this was 63,788 as of Wed at 10pm.  When it changets, get new ones overnight.  This step is v slow.

In [6]:
products = Observations.get_product_list(obs)
# Note how many data product files were included
len(products)

63662

In [ ]:
#obs.keys()

## Data Product Retrieval

Products may be selected for download to your local machine. Note that you will need to login with a valid MAST.auth token to download exclusive access (EAP) data. Get one from https://auth.mast.stsci.edu/info. The following will prompt for a token.

<div class="alert alert-block alert-info">

<span style="color:black">
Typing tokens into a terminal or a Jupyter notebook is a security risk. Instead, input the token using a more secure method, as illustrated below with <b>getpass</b>.
    
</span>
</div>

In [6]:
import getpass as gp
Observations.login(token=gp.getpass(prompt='Enter MAST.auth token: '))

Enter MAST.auth token: ········
INFO: MAST API token accepted, welcome Jane Rigby [astroquery.mast.auth]


### Product Filtering

Next, select L-1b products (i.e., the UNCALibrated products, with the file suffix _uncal.fits) from the product Table for retrieval. Then verify that the result is what you expect. 

In [ ]:
#PICK ONE OF THE NEXT 2 CELLS!  They make different choices. Haven't figured out how to get both yet.

In [11]:
# JRR added "CAL" next to UNCAL, to get both _cal and _uncal files.
manifest = Observations.filter_products(
           products
        ,productSubGroupDescription=('I2D'))#   ('WFSCMB'))  #('CAL', 'RATE', 'JSON')
manifest['proposal_id','productFilename','calib_level']

proposal_id,productFilename,calib_level
str4,str52,int64
1448,jw01448-c1001_t014_niriss_clear-f115w_i2d.fits,3
1448,jw01448-c1001_t014_niriss_clear-f200w_i2d.fits,3
1448,jw01448-c1002_t003_niriss_clear-f115w_i2d.fits,3
1448,jw01448-c1002_t003_niriss_clear-f200w_i2d.fits,3
1448,jw01448-c1003_t015_niriss_clear-f115w_i2d.fits,3
1448,jw01448-c1003_t015_niriss_clear-f200w_i2d.fits,3
1448,jw01448-c1004_t005_niriss_clear-f115w_i2d.fits,3
1448,jw01448-c1004_t005_niriss_clear-f200w_i2d.fits,3
1448,jw01448-c1005_t006_niriss_clear-f115w_i2d.fits,3


### Retrieval

Now download the filtered products to your local platform. Setting the optional **curl_flag** parameter to **True** will instead download a bash script that contains cURL commands to fetch the files at a later time. This approach is highly recommended for large numbers of files. 

<div class="alert alert-block alert-info">

<span style="color:black">
    <b>Note:</b> this is not an efficient operation in a notebook; it may be better to run this whole example from a python session or a script.

</span>
</div>

In [12]:
download_bundle = Observations.download_products(
           manifest
           ,curl_flag=True
           )